## Description
This notebook generates the data for the shortest problem. We use m-by-m grid graphs and a vertex-weighted version of the shortest path problem, following Vastelica et al and Berthet et al. The data consists of:
  - pairs (d,x) where d is a five dim contextual vector and x encodes the shortest path.
  - W, a d-by-m^2 matrix which maps contexts to vertex weights.
  - other stuff

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys

sys.path.append('/content/drive/MyDrive/Projects/2022-SPO-with-DYS')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd drive/MyDrive/Projects/2022-SPO-with-DYS

In [1]:
import torch
from GenerateShortestPathData import create_shortest_path_data
import time

ModuleNotFoundError: No module named 'GenerateShortestPathData'

In [ ]:
## Fix some parameters
train_size = 10
test_size = 2
context_size = 5
# m_max = 25 # generate m-by-m graphs, with m in increments of 5, between 5 and m_max inclusive. 

In [ ]:
# Loop over graph sizes
# grid_array = [5,10,15,20,25,50,100]
# for m in grid_array:
m = 100
start_time = time.time()
print('Start building dataset for m = ' + str(m))
train_dataset_v, test_dataset_v, train_dataset_e, test_dataset_e, WW, A, b, num_edges, Edge_list = create_shortest_path_data(m, train_size, test_size, context_size)
end_time = time.time()
print('Created data! Now saving')
state = {
      'WW': WW,
      'train_dataset_v': train_dataset_v,
      'test_dataset_v': test_dataset_v,
      'train_dataset_e': train_dataset_e,
      'test_dataset_e': test_dataset_e,
      'm': m,
      'A':A,
      'b':b,
      'num_edges': num_edges,
      'Edge_list': Edge_list,
      }
save_dir = './shortest_path_data/'
state_path = save_dir + 'Shortest_Path_training_data' + str(m) +'.pth'
torch.save(state, state_path)
print('Finished building dataset for m = ' + str(m), ', Time = ', end_time - start_time, ' seconds')

## Examine some paths
The next two blocks are test code to see if the new data generating functions are working.

In [ ]:
## Examine some paths
state = torch.load('./shortest_path_data/Shortest_Path_training_data5.pth')

## Extract data from state
train_dataset_e = state['train_dataset_e']
test_dataset_e = state['test_dataset_e']
train_dataset_v = state['train_dataset_v']
test_dataset_v = state['test_dataset_v']
m = state["m"]
A = state["A"].float()
b = state["b"].float()
num_edges = state["num_edges"]
Edge_list = state["Edge_list"]
Edge_list_torch = torch.tensor(Edge_list)

## Visualization
The next few blocks help visualize the outputs. Just dumping them in this notebook temporarily.

In [ ]:
from source.utils import Edge_to_Node
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

train_loader_e = DataLoader(dataset=train_dataset_e, batch_size=200,
                                  shuffle=False)
d_batch, path_batch = next(iter(train_loader_e))

train_loader_v = DataLoader(dataset=train_dataset_v, batch_size=200,
                                  shuffle=False)
d_batch_v, path_batch_v = next(iter(train_loader_v))

for i in range(98,100):
    # print(Edge_to_Node(path_batch[i], Edge_list, m,'cpu'))
    plt.imshow(path_batch_v[i])
    plt.show()
#     plt.imshow(Edge_to_Node(path_batch[i], Edge_list, m,'cpu') - path_batch_v[i])
#     plt.show()

In [ ]:
import matplotlib.pyplot as plt
d_batch, path_batch = next(iter(test_loader))

d_batch = d_batch.to(device)
path_batch =path_batch.to(device)
path_pred = net(d_batch)

for i in range(10):
  node_map = Edge_to_Node(path_batch[i], Edge_list, m,'cuda:0')
  plt.matshow(node_map.to('cpu'))
  # plt.savefig('10_by_10_True_path'+str(i)+'.pdf')
  plt.show()
  node_map_pred = Edge_to_Node(path_pred[i], Edge_list, m,'cuda:0')
  plt.matshow(node_map_pred.to('cpu'))
  plt.savefig('10_by_10_Pred_path'+str(i)+'.pdf')
  # print(path_pred[i])